**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [2]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [3]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

169001437/169001437 [==============================] - 573s 3us/step


Подробрнее о датасете можно прочитать тут - https://www.kaggle.com/datasets/fedesoriano/cifar100?select=train

Если кратко, то датасет на 60000 фотографий. Изображения делятся на 100 классов по 600 изображений в каждом.

In [4]:
# Преобразуйте метки классов в one_hot формат
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
y_train = enc.fit_transform(y_train)
y_val = enc.fit_transform(y_val)

In [5]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [191]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(8, 2, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),
    
    tf.keras.layers.Conv2D(16, 2, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),
    
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [192]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6432e6


In [193]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_193 (Conv2D)         (None, 32, 32, 8)         104       
                                                                 
 batch_normalization_187 (Ba  (None, 32, 32, 8)        32        
 tchNormalization)                                               
                                                                 
 max_pooling2d_173 (MaxPooli  (None, 15, 15, 8)        0         
 ng2D)                                                           
                                                                 
 conv2d_194 (Conv2D)         (None, 15, 15, 16)        528       
                                                                 
 batch_normalization_188 (Ba  (None, 15, 15, 16)       64        
 tchNormalization)                                               
                                                     

In [194]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=10000, decay_rate=0.2)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [198]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=30
)

Epoch 1/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2535 - accuracy: 0.4350 - val_loss: 2.9360 - val_accuracy: 0.3112
Epoch 2/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2472 - accuracy: 0.4342 - val_loss: 2.9365 - val_accuracy: 0.3094
Epoch 3/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2410 - accuracy: 0.4370 - val_loss: 2.9319 - val_accuracy: 0.3147
Epoch 4/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2356 - accuracy: 0.4389 - val_loss: 2.9207 - val_accuracy: 0.3168
Epoch 5/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2313 - accuracy: 0.4396 - val_loss: 2.9478 - val_accuracy: 0.3164
Epoch 6/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2236 - accuracy: 0.4430 - val_loss: 2.9335 - val_accuracy: 0.3147
Epoch 7/30
391/391 [==============================] - 11s 28ms/step - loss: 2.2215 - accuracy: 0.4426 - val_loss: 2.9278 - val_accuracy:

In [199]:
scores = model.evaluate(X_val, y_val)
print(f"Accuracy: {(scores[1]*100):.2f}%")

313/313 [==============================] - 1s 4ms/step - loss: 2.9665 - accuracy: 0.3170
Accuracy: 31.70%
